In [1]:
import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)
 
# 切换到上一层目录
parent_dir = os.path.dirname(current_dir)
os.chdir(parent_dir)
print("切换后的目录：", parent_dir)

当前工作目录： c:\Users\zhaoxs3\Desktop\ValLib\unit_test
切换后的目录： c:\Users\zhaoxs3\Desktop\ValLib


In [2]:
import QuantLib as ql
import pandas as pd

from devlib.utils.curve_utils import get_ois_float_rate, get_real_fixing_date
from devlib.market.curves.overnight_index_curves import Sofr
from devlib.products.rates.irs.general_irs import *

In [3]:
from vallib.products.rates import *

####################################################################
#  ValLib ALPHA Version 0.1.0 - This build: 24 Jan 2025 at 10:42 #
####################################################################



In [4]:
today = ql.Date(27, 11, 2024)
ql.Settings.instance().evaluationDate = today

mkt_file_path = './unit_test/data/sofr_curve_data_20241127.xlsx'
swap_mkt_data = pd.read_excel(mkt_file_path, sheet_name='swap')
fixing_data = pd.read_excel(mkt_file_path, sheet_name='fixing')

index_curve = Sofr(today, swap_mkt_data=swap_mkt_data, fixing_data=fixing_data)
discount_curve = Sofr(today, swap_mkt_data=swap_mkt_data, fixing_data=fixing_data)

In [5]:
value_dt = Date(27,11,2024)
curve = OISCurve.build_curve_from_ql(value_dt, discount_curve.curve)

In [6]:
curve.print_table(payment_dt=Date(27,12,2024))

TypeError: No matching definition for argument type(s) float64, reflected list(float64)<iv=None>, reflected list(float64)<iv=None>, int64

In [6]:
discount_curve.curve.dates()

(Date(27,11,2024),
 Date(9,12,2024),
 Date(16,12,2024),
 Date(23,12,2024),
 Date(2,1,2025),
 Date(3,2,2025),
 Date(3,3,2025),
 Date(2,4,2025),
 Date(2,5,2025),
 Date(2,6,2025),
 Date(2,7,2025),
 Date(4,8,2025),
 Date(2,9,2025),
 Date(2,10,2025),
 Date(3,11,2025),
 Date(2,12,2025),
 Date(4,6,2026),
 Date(4,12,2026),
 Date(6,12,2027),
 Date(6,12,2028),
 Date(5,12,2029),
 Date(4,12,2030),
 Date(4,12,2031),
 Date(6,12,2032),
 Date(6,12,2033),
 Date(6,12,2034),
 Date(4,12,2036),
 Date(6,12,2039),
 Date(6,12,2044),
 Date(6,12,2049),
 Date(4,12,2054),
 Date(4,12,2064),
 Date(5,12,2074))

In [7]:
ccy = 'USD'
index_name = 'SOFR'
sch_calendar = ql.UnitedStates(ql.UnitedStates.FederalReserve)
fixing_calendar = ql.Sofr().fixingCalendar()    
payment_calendar = ql.UnitedStates(ql.UnitedStates.FederalReserve)
date_generation_rule = ql.DateGeneration.Backward
payment_delay = 0
fixing_days = 0
end_of_month = True
is_ois_leg = True

effective_date = ql.Date(19, 7, 2024)
# maturity_date = sch_calendar.advance(effective_date, ql.Period('1Y'), sch_convention)
maturity_date = ql.Date(18, 7, 2025)
fixed_leg_pay_freq = '1Y'
float_leg_pay_freq = '1Y'
reset_freq = 'None'
notional = 32061781.09
fixed_rate = 0.05558
multiplier = 1
spread = 0
fixed_leg_daycount = ql.Actual360()
float_leg_daycount = ql.Actual360()
fixed_leg_direction = 'pay'
float_leg_direction = 'receive'

In [8]:
inst = StandardFloatFixedIrs(
    ccy, index_name, effective_date, maturity_date, fixed_leg_pay_freq, 
    float_leg_pay_freq, reset_freq, notional, fixed_rate, multiplier, spread, 
    fixed_leg_daycount, float_leg_daycount, payment_delay, fixing_days, 
    sch_calendar, fixing_calendar, payment_calendar, fixed_leg_direction, 
    float_leg_direction, date_generation_rule=date_generation_rule, 
    end_of_month=end_of_month, is_ois_leg=is_ois_leg)

In [9]:
inst.float_leg.start_dates

array([Date(19,7,2024)], dtype=object)

In [10]:
inst.float_leg.end_dates

array([Date(18,7,2025)], dtype=object)

In [ ]:
get_ois_float_rate(index_curve, today, inst.float_leg.start_dates[0], inst.float_leg.end_dates[0], ql.Actual360(), spread)

np.float64(0.04674658104292555)

In [12]:
calendar = ql.UnitedStates(ql.UnitedStates.FederalReserve)

In [13]:
last_reset_date = calendar.advance(inst.float_leg.end_dates[0], ql.Period('-1D'))

In [14]:
last_fixing_date = get_real_fixing_date(index_curve, last_reset_date)

In [15]:
today

Date(27,11,2024)

In [16]:
calendar.adjust(today, ql.Preceding)

Date(27,11,2024)

In [17]:
def get_forward_rate(index_curve, today, fixing_date, use_last_fixing=False, product_type='linear'):
    if today < index_curve.today:
        raise Exception('Valuation date should not be earlier than curve date!')
    
    try:
        rate = index_curve.get_forward_rate(today, fixing_date)
    except:
        index = index_curve.index
        curve= index_curve.curve
        freq = index.tenor()
        calendar = index.fixingCalendar()
        daycount = index.dayCounter()
        convention = index.businessDayConvention()
        end_of_month = index.endOfMonth()
        
        # settlement_delay = 0 
        settlement_delay = index.fixingDays()
        default_compounding_method = ql.Simple
        
        real_fixing_date = get_real_fixing_date(index_curve, fixing_date, product_type=product_type)
            
        if real_fixing_date <= today:
            if real_fixing_date <= index_curve.today:
                # history fixing data exist
                rate = index.fixing(real_fixing_date)
            else:
                # real_fixing_date > index_curve.today, so fixing data does not exist
                if use_last_fixing:
                    last_fixing_date = calendar.adjust(index_curve.today, ql.Preceding)
                    rate = index.fixing(last_fixing_date)
                else:
                    real_date = calendar.advance(
                        real_fixing_date, ql.Period(settlement_delay, ql.Days))
                    real_end_date = calendar.advance(real_date, freq, convention, end_of_month)
                    rate = curve.forwardRate(
                        real_date, real_end_date, daycount, default_compounding_method).rate()
        else:
            real_date = calendar.advance(
                real_fixing_date, ql.Period(settlement_delay, ql.Days))
            real_end_date = calendar.advance(real_date, freq, convention, end_of_month)
            rate = curve.forwardRate(
                real_date, real_end_date, daycount, default_compounding_method).rate()
                
    # print(f'fixing date: {str(fixing_date.to_date())}, reset rate: {rate}')
    
    return rate

In [18]:
def get_fixing_rates(index_curve, today, fixing_dates, use_last_fixing=False, product_type='linear'):
    rate_list = []
    for fixing_date in fixing_dates:
        rate = get_forward_rate(index_curve, today, fixing_date, 
                                use_last_fixing=use_last_fixing, product_type=product_type)
        rate_list.append(rate)
    
    return np.array(rate_list)

In [19]:
fixed_dates = np.array([today])

In [20]:
fixed_dates = np.array(
    ql.Schedule(inst.float_leg.start_dates[0], today, ql.Period('1D'), 
                calendar, ql.Following, ql.Preceding, 
                ql.DateGeneration.Forward, False))

In [21]:
fixed_rates = get_fixing_rates(index_curve, today, fixed_dates, use_last_fixing=False)

In [22]:
fixed_rates

array([0.0534, 0.0533, 0.0534, 0.0534, 0.0535, 0.0535, 0.0533, 0.0533,
       0.0538, 0.0535, 0.0535, 0.0532, 0.0533, 0.0533, 0.0534, 0.0534,
       0.0534, 0.0534, 0.0533, 0.0535, 0.0532, 0.0532, 0.0532, 0.0531,
       0.0531, 0.0533, 0.0534, 0.0535, 0.0535, 0.0533, 0.0532, 0.0534,
       0.0535, 0.0535, 0.0534, 0.0534, 0.0533, 0.0532, 0.0533, 0.0533,
       0.0538, 0.0538, 0.0533, 0.0482, 0.0483, 0.0483, 0.0484, 0.0484,
       0.0483, 0.0484, 0.0496, 0.0505, 0.0492, 0.0485, 0.0483, 0.0483,
       0.0484, 0.0483, 0.0482, 0.0481, 0.0486, 0.0486, 0.0485, 0.0484,
       0.0482, 0.0483, 0.0483, 0.0483, 0.0483, 0.0482, 0.0482, 0.0481,
       0.049 , 0.0486, 0.0482, 0.0482, 0.0481, 0.0482, 0.046 , 0.046 ,
       0.0459, 0.0458, 0.0457, 0.0457, 0.0457, 0.0456, 0.0457, 0.0457,
       0.0458, 0.0458, 0.0457])

In [23]:
next_reset_date = calendar.advance(today, ql.Period('1D'))
future_forward_rate = index_curve.curve.forwardRate(
    next_reset_date, inst.float_leg.end_dates[0], ql.Actual360(), ql.Simple).rate()
rate_array = np.append(fixed_rates, future_forward_rate)
reset_dates = np.append(fixed_dates, [next_reset_date, inst.float_leg.end_dates[0]])

In [24]:
reset_period_dcf = np.array(
            [ql.Actual360().yearFraction(reset_dates[i], reset_dates[i+1]) 
                for i in range(np.size(reset_dates) - 1)])
float_rate = ((np.prod(rate_array * reset_period_dcf + 1) - 1)
                / np.sum(reset_period_dcf))

In [25]:
reset_period_dcf

array([0.00833333, 0.00277778, 0.00277778, 0.00277778, 0.00277778,
       0.00833333, 0.00277778, 0.00277778, 0.00277778, 0.00277778,
       0.00833333, 0.00277778, 0.00277778, 0.00277778, 0.00277778,
       0.00833333, 0.00277778, 0.00277778, 0.00277778, 0.00277778,
       0.00833333, 0.00277778, 0.00277778, 0.00277778, 0.00277778,
       0.00833333, 0.00277778, 0.00277778, 0.00277778, 0.00277778,
       0.01111111, 0.00277778, 0.00277778, 0.00277778, 0.00833333,
       0.00277778, 0.00277778, 0.00277778, 0.00277778, 0.00833333,
       0.00277778, 0.00277778, 0.00277778, 0.00277778, 0.00833333,
       0.00277778, 0.00277778, 0.00277778, 0.00277778, 0.00833333,
       0.00277778, 0.00277778, 0.00277778, 0.00277778, 0.00833333,
       0.00277778, 0.00277778, 0.00277778, 0.00277778, 0.01111111,
       0.00277778, 0.00277778, 0.00277778, 0.00833333, 0.00277778,
       0.00277778, 0.00277778, 0.00277778, 0.00833333, 0.00277778,
       0.00277778, 0.00277778, 0.00277778, 0.00833333, 0.00277

In [26]:
float_rate

np.float64(0.04674658104292555)

In [27]:
cal = ql.UnitedStates(ql.UnitedStates.FederalReserve)

In [28]:
cal.holidayList(ql.Date(1,1,2028), ql.Date(31,12,2028))

(Date(17,1,2028),
 Date(21,2,2028),
 Date(29,5,2028),
 Date(19,6,2028),
 Date(4,7,2028),
 Date(4,9,2028),
 Date(9,10,2028),
 Date(23,11,2028),
 Date(25,12,2028))

In [29]:
cal.isHoliday(ql.Date(3,6,2026))

False